In [ ]:
!pip3 install scikit-video

     |████████████████████████████████| 2.3 MB 14.0 MB/s 


In [ ]:


import numpy as np
import cv2
from skimage.morphology import disk, opening, remove_small_objects, label
import skvideo.io

from google.colab import drive
drive.mount('/content/drive', force_remount=True)


def normalize0_255(X):
	X = (X - X.min()) / (X.max() - X.min())
	return np.array(X * 255, dtype=np.uint8)

def matrixDivision(M, N):
    C = np.divide(M, N, out=np.zeros_like(M), where=N != 0)
    return C

def order_points(pts):
	# initialzie a list of coordinates that will be ordered
	# such that the first entry in the list is the top-left,
	# the second entry is the top-right, the third is the
	# bottom-right, and the fourth is the bottom-left
	rect = np.zeros((4, 2), dtype = "float32")
	# the top-left point will have the smallest sum, whereas
	# the bottom-right point will have the largest sum
	s = pts.sum(axis = 1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]
	# now, compute the difference between the points, the
	# top-right point will have the smallest difference,
	# whereas the bottom-left will have the largest difference
	diff = np.diff(pts, axis = 1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]
	# return the ordered coordinates
	return rect

def four_point_transform(image, pts):
	# obtain a consistent order of the points and unpack them
	# individually
	rect = order_points(pts)
	(tl, tr, br, bl) = rect
	# compute the width of the new image, which will be the
	# maximum distance between bottom-right and bottom-left
	# x-coordiates or the top-right and top-left x-coordinates
	widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
	widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
	maxWidth = max(int(widthA), int(widthB))
	# compute the height of the new image, which will be the
	# maximum distance between the top-right and bottom-right
	# y-coordinates or the top-left and bottom-left y-coordinates
	heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
	heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
	maxHeight = max(int(heightA), int(heightB))
	# now that we have the dimensions of the new image, construct
	# the set of destination points to obtain a "birds eye view",
	# (i.e. top-down view) of the image, again specifying points
	# in the top-left, top-right, bottom-right, and bottom-left
	# order
	dst = np.array([
		[0, 0],
		[maxWidth - 1, 0],
		[maxWidth - 1, maxHeight - 1],
		[0, maxHeight - 1]], dtype = "float32")
	# compute the perspective transform matrix and then apply it
	M = cv2.getPerspectiveTransform(rect, dst)
	warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
	# return the warped image
	return warped


def summarizeHorizontal(linhaHorizontal):
  h = np.empty(len(linhaHorizontal), dtype=int)
  idx = 0
  while(idx < len(linhaHorizontal)):
    if linhaHorizontal[idx] == 0:
      h[idx] = 0
      idx = idx + 1
    else:
      i_start = idx
      soma = 0
      while idx < len(linhaHorizontal) and linhaHorizontal[idx]==1:
        soma = soma + linhaHorizontal[idx]
        idx = idx + 1
      i_end = idx
      h[i_start:i_end] = soma
  return h

pvideo = cv2.VideoCapture('/content/drive/My Drive/PFC/MVI_2576.MOV')


kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))

# Parametro; quantidade de frames para operação AND lógico
historyFrames = 5 # artigo usa 8, antes era 5
listPrevFrames = []
index = 0
contFrames = 0
#writerCulturaErvas = cv2.VideoWriter("/content/drive/My Drive/AnaPaula/dados/CulturaErvas.mkv", cv2.VideoWriter_fourcc(*'X264'), 30.0, (640,360))
#writer = skvideo.io.FFmpegWriter("/content/drive/My Drive/AnaPaula/dados/outputvideo.mp4")


while(pvideo.isOpened()):
    if index%10 ==0:
      print("Processando frame = ", index)
    ret, frame = pvideo.read()
    frame_lido = np.copy(frame)

    if not ret or index >= 5800:
      print("Fim de processamento. Saindo ...")
      break
    if index <= 1300:
      index = index + 1
      continue
    #frame = cv2.resize(frame,None,fx=0.5,fy=0.5)
    
    frame = frame[280:640,640:1280,:] # 360 x 640
    frame_lido = cv2.rectangle(frame_lido,(640, 280),(1280, 640),(0,0,255),3)

    frame_recortado = np.copy(frame)
    pts = np.float32([[240, 0],[440,0],[0,359],[639,359]])
    framePerspectiva = four_point_transform(frame, pts)
    frame = framePerspectiva

    frame_original = np.copy(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    cultura_erva = np.copy(frame_original)
    FIPSobreposto = np.copy(frame_original)
    FIPRCRCSobreposto = np.copy(frame_original)


    frame = frame/255
    R = frame[:,:,0]
    G = frame[:,:,1]
    B = frame[:,:,2]

    RGB = R + G + B

    r = matrixDivision(R,RGB)
    g = matrixDivision(G,RGB)
    b = matrixDivision(B,RGB)

    CIVE = 0.441*r - 0.881*g + 0.385*b + 18.78745
    CIVE_norm = normalize0_255(CIVE)

    ret2,binOtsu = cv2.threshold(CIVE_norm,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    binOtsu = (255 - binOtsu)/255 

    if contFrames < historyFrames:
    	listPrevFrames.append(binOtsu)
    else:
    	listPrevFrames.pop(0)
    	listPrevFrames.append(binOtsu)
    contFrames = contFrames + 1
    
    
    
    resAND = np.ones((CIVE_norm.shape[0], CIVE_norm.shape[1]))
    for binImg in listPrevFrames:
    	resAND = np.logical_and(resAND, binImg)
          
    # Fazer operacao de abertur morfológica (opening): verificar tamanho do elemento estruturante ...
    # usado no artigo do Burgos
    #resOpening = cv2.morphologyEx(resAND.astype(np.uint8), cv2.MORPH_OPEN, kernel)
    resOpening = cv2.erode(resAND.astype(np.uint8), kernel)


    # Extracao das linhas de plantio (areas de 	7 a 10 vezes maiores que as manchas de ervas daninhas). 
    # Como estimar isso se não seu o tamanho das áreas de ervas daninhas. Talvez ele explica mais adiante nos resultados
    # Resumindo: Remover regioes menores que um dado limiar
    resOpening_bool = np.array(resOpening, bool) 
    resOpening_cleaned = remove_small_objects(resOpening_bool, min_size=500)
    resOpening_bool = np.array(resOpening, bool) 
    RCRD = np.array(resOpening_cleaned, dtype='int')

    #FIM##############################################################################################


    # Fast Image Processing (FIP) de Baixa Granuralidade
    #INICIO##########################################################################################
    lenFatia = 6 # o artigo usa 16. Antes era 8 #[4, 8, 12, 16]
    thrFatia = 4 #round(0.8*lenFatia) # o artigo usa 12 (que é 3/4 de 16). Antes era 5 # 
    thrLargura = 3.0
    #ajuste_thrLargura = 0.1

    [nLines, nColumns] = binOtsu.shape
    nFatias = int(nLines/lenFatia)
    PROJImage = np.zeros((nLines, nColumns))
    FIPImage = np.zeros((nLines, nColumns))
    cultureImage = np.zeros((nLines, nColumns))
    ervasDaninhas = np.zeros((nLines, nColumns))
    for i in range(nFatias)[::-1]: # de nFatias - 1 até zero
    	p = np.sum(binOtsu[i*lenFatia:i*lenFatia+lenFatia],axis=0) #soma ao longo de fatia
    	p = p>=thrFatia # filtra a projecao
        
    	# escreve a projecao vertical numa imagem preta e branco  
    	PROJImage[i*lenFatia:i*lenFatia+lenFatia] = np.array(p, dtype='int')
       
      # Fazer filtro com limiar de 5% da largura da imagem (linha), na fatia de baixo. Diminuar o limiar em 2.5% ao longo de cada fatia acima, isto é ...  
    	h = summarizeHorizontal(np.array(p, dtype='int'))
       
        
    	#thrLargura = thrLargura+ thrLargura*ajuste_thrLargura
    	FIPImage[i*lenFatia:i*lenFatia+lenFatia] = np.array((h > thrLargura), dtype='int')
    FIPSobreposto[np.array(FIPImage, dtype='bool')] = [0, 255, 0] #[blue, green, red]

    FIPRCRCSobreposto[np.array(FIPImage, dtype='bool')] = [0, 255, 0]
    FIPRCRCSobreposto[resOpening_cleaned] = [0, 0, 255]



    # Fast Image Processing (FIP) de Alta Granuralidade
    #INICIO##########################################################################################
    lenFatia = 86 # o artigo usa 16. Antes era 8
    thrFatia = 54 #round(0.8*lenFatia) # o artigo usa 12 (que é 3/4 de 16). Antes era 5
    thrLargura = 3.0
    #ajuste_thrLargura = 0.1

    [nLines, nColumns] = binOtsu.shape
    nFatias = int(nLines/lenFatia)
    PROJImage_ALTA = np.zeros((nLines, nColumns))
    FIPImage_ALTA = np.zeros((nLines, nColumns))
    ervasDaninhas_COLADAS = np.zeros((nLines, nColumns))
    for i in range(nFatias)[::-1]: # de nFatias - 1 até zero
    	p = np.sum(binOtsu[i*lenFatia:i*lenFatia+lenFatia],axis=0) #soma ao longo de fatia
    	p = p>=thrFatia # filtra a projecao
        
    	# escreve a projecao vertical numa imagem preta e branco  
    	PROJImage_ALTA[i*lenFatia:i*lenFatia+lenFatia] = np.array(p, dtype='int')
       
      # Fazer filtro com limiar de 5% da largura da imagem (linha), na fatia de baixo. Diminuar o limiar em 2.5% ao longo de cada fatia acima, isto é ...  
    	h = summarizeHorizontal(np.array(p, dtype='int'))
       
        
    	#thrLargura = thrLargura+ thrLargura*ajuste_thrLargura
    	FIPImage_ALTA[i*lenFatia:i*lenFatia+lenFatia] = np.array((h > thrLargura), dtype='int')
     	

    #FIM##########################################################################################

    # JUNCAO DE RCRD COM FIP

    # VEGETACAO = regiões de FIP que tem interseção com a imagem RCRD
    # ideia: rotular as regiões de FIP. Para cada regiões de FIP verificar se a intersecção com RCRD é não vazia
    labeledFIP =   label(binOtsu, connectivity=2) # connectivity=2 eh o mesmo que 8-conectados
    for lab in range(1, labeledFIP.max()+1):
    	regiaoSel = labeledFIP==lab
    	resIntesection = np.logical_and(regiaoSel, resOpening_cleaned)
    	if not np.all(resIntesection == False):
    	  cultura_erva[regiaoSel] = [0, 255, 0] # desenha na imagem original
    	  cultureImage[regiaoSel] = 1

    # ERVAS DANINHAS = todos os pixels de vegetacao que nao sao cultura (linha de plantion)
    ervasDaninhas = binOtsu - cultureImage # cultureImage eh linha de plantio 
    

    # Calcula ervas daninhas coladas
    ervasDaninhas_COLADAS = np.array(binOtsu,int) - np.array(FIPImage_ALTA, dtype='int')
    ervasDaninhas_COLADAS1 = ervasDaninhas_COLADAS > 0
    ervasDaninhas_COLADAS2 = remove_small_objects(ervasDaninhas_COLADAS1, min_size=800)

    ervasDaninhasCOPY = np.copy(ervasDaninhas)

    ervasDaninhas = np.logical_or(ervasDaninhas, ervasDaninhas_COLADAS2)

    # EXCLUE ERVAS DANINHAS NAS MARGENS ESQUERDA E DIREITA
    [N,M] = ervasDaninhas.shape
    #print(N, M)
    coords = np.argwhere(ervasDaninhas == True)
    for coord in coords:
      if coord[1]<=100 or abs(coord[1]-M)<=100 or abs(coord[0]-N)<=50:
        ervasDaninhas[coord[0], coord[1]] = False

    cultura_erva[np.array(ervasDaninhas, bool)] = [0, 0, 255]

    linhasCompleta = cultureImage - ervasDaninhas


    #Assim que processar cada frame
    #writerCulturaErvas.write(cultura_erva)
    #writer.writeFrame(cv2.cvtColor(cultura_erva, cv2.COLOR_BGR2RGB))


    if index%25 == 0:
      #cv2.imwrite('/content/drive/My Drive/AnaPaula/dados/saidav2/frame'+str(index)+'.png',frame_original)
      cv2.imwrite('/content/drive/My Drive/PFC/saidav3/frame'+str(index)+'LIDO.png',frame_lido)
      cv2.imwrite('/content/drive/My Drive/PFC/saidav3/frame'+str(index)+'FIP_ALTA.png',FIPImage_ALTA*255)
      cv2.imwrite('/content/drive/My Drive/PFC/saidav3/frame'+str(index)+'ERVASBURGOS.png',ervasDaninhasCOPY*255)
      cv2.imwrite('/content/drive/My Drive/PFC/saidav3/frame'+str(index)+'RESIDUO.png',ervasDaninhas_COLADAS1*255)
      cv2.imwrite('/content/drive/My Drive/PFC/saidav3/frame'+str(index)+'ERVAS_COLADAS.png',ervasDaninhas_COLADAS2*255)
      cv2.imwrite('/content/drive/My Drive/PFC/saidav3/frame'+str(index)+'RECORTADO.png',frame_recortado) 
      cv2.imwrite('/content/drive/My Drive/PFC/saidav3/frame'+str(index)+'Perspectiva.png',framePerspectiva)
      cv2.imwrite('/content/drive/My Drive/PFC/saidav3/frame'+str(index)+'CIVE.png',CIVE_norm)
      cv2.imwrite('/content/drive/My Drive/PFC/saidav3/frame'+str(index)+'OTSU.png',binOtsu*255)
      cv2.imwrite('/content/drive/My Drive/PFC/saidav3/frame'+str(index)+'AND.png',resAND*255)
      cv2.imwrite('/content/drive/My Drive/PFC/saidav3/frame'+str(index)+'PROJVertical.png',PROJImage*255)
      cv2.imwrite('/content/drive/My Drive/PFC/saidav3/frame'+str(index)+'FIP.png',FIPImage*255)
      cv2.imwrite('/content/drive/My Drive/PFC/saidav3/frame'+str(index)+'FIPSobreposto.png',FIPSobreposto)
      cv2.imwrite('/content/drive/My Drive/PFC/saidav3/frame'+str(index)+'FIPRCRDSobreposto.png',FIPRCRCSobreposto)
      cv2.imwrite('/content/drive/My Drive/PFC/saidav3/frame'+str(index)+'Opening.png',resOpening*255)
      cv2.imwrite('/content/drive/My Drive/PFC/saidav3/frame'+str(index)+'Opening_Cleaned[RCRD].png',RCRD*255)
      cv2.imwrite('/content/drive/My Drive/PFC/saidav3/frame'+str(index)+'CulturaErvas.png',cultura_erva)
      cv2.imwrite('/content/drive/My Drive/PFC/saidav3/frame'+str(index)+'ErvasDaninhas.png',ervasDaninhas*255)
      cv2.imwrite('/content/drive/My Drive/PFC/saidav3/frame'+str(index)+'LinhasCultura.png',cultureImage*255)
      cv2.imwrite('/content/drive/My Drive/PFC/saidav3/frame'+str(index)+'LinhasCulturaCompletas.png',linhasCompleta*255)
    index = index + 1

pvideo.release()
#writerCulturaErvas.release()
#writer.close()
#cv2.destroyAllWindows()




Mounted at /content/drive
Processando frame =  0
Processando frame =  10
Processando frame =  20
Processando frame =  30
Processando frame =  40
Processando frame =  50
Processando frame =  60
Processando frame =  70
Processando frame =  80
Processando frame =  90
Processando frame =  100
Processando frame =  110
Processando frame =  120
Processando frame =  130
Processando frame =  140
Processando frame =  150
Processando frame =  160
Processando frame =  170
Processando frame =  180
Processando frame =  190
Processando frame =  200
Processando frame =  210
Processando frame =  220
Processando frame =  230
Processando frame =  240
Processando frame =  250
Processando frame =  260
Processando frame =  270
Processando frame =  280
Processando frame =  290
Processando frame =  300
Processando frame =  310
Processando frame =  320
Processando frame =  330
Processando frame =  340
Processando frame =  350
Processando frame =  360
Processando frame =  370
Processando frame =  380
Processand